## MNIST with CNN 

In [36]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [37]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Functions

Function to help initialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [38]:
# INIT WEIGHTS
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [39]:
# INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF.

Compute a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter/kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op performs the 
following:

1. Flattens the filter to a 2-D matrix with shape
    `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual* tensor of shape `[batch, out_height, out_width, filter_height*filter_width*in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch vector.

In [40]:
# CONV2D
def conv2d(x, W):
    """
     x --> [batch, H, W, Channels]
     W --> [filter H, filter W, Channels IN, Channels OUT]
    """
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

Create a max pooling layer, again using built in TF functions:
performs the max pooling on the input.

Args: 
- values: A 4-D `Tensor` with shape `[batch, height, width, channels]` and 
    type `tf.float32`.
- ksize: A list of ints that has length >= 4. The size of the window for 
    each dimension of the input tensor.
- strides: A list of ints that has length >= 4. The stride of the sliding window for each dimension of the input tensor.
- padding: A string, either `'VALID'` or `'SAME'`.

In [41]:
# POOLING
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Using the conv2d function, we'll return an actual convolutional layer here that 
uses an ReLu activation.

In [42]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    
    return tf.nn.relu(conv2d(input_x, W) + b)

In [43]:
# NORMAL (FULLY CONNECTED) LAYER
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### PlaceHolders 

In [44]:
x = tf.placeholder(tf.float32, shape=[None, 784])

In [45]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

### Layers

In [46]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [47]:
convo_1 = convolutional_layer(x_image, shape=[5, 5, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

In [48]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [49]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])

In [50]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [51]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [52]:
y_pred = normal_full_layer(full_one_dropout, 10)

### Loss Function

In [53]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                                                      logits=y_pred)) 

### Optimizer

In [54]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

### Initialize Variables

In [ ]:
init = tf.global_variables_initializer()

### Session 

In [ ]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1601


Currently on step 100
Accuracy is:
0.8339


Currently on step 200
Accuracy is:
0.9083


Currently on step 300
Accuracy is:
0.9241


Currently on step 400
Accuracy is:
0.9401


Currently on step 500
Accuracy is:
0.9478


Currently on step 600
Accuracy is:
0.9519


Currently on step 700
Accuracy is:
0.9552


Currently on step 800
Accuracy is:
0.956


Currently on step 900
Accuracy is:
0.9626


Currently on step 1000
Accuracy is:
0.9658


Currently on step 1100
Accuracy is:
0.9678


Currently on step 1200
Accuracy is:
0.967


Currently on step 1300
Accuracy is:
0.9679


Currently on step 1400
Accuracy is:
0.9709


